# Introduction 

Romance only

In [1]:
%matplotlib inline
import os
import sys
from math import exp
import pickle
from importlib import reload
from collections import deque

In [2]:
import numpy as np
from sklearn.externals import joblib
import random
from tqdm import tqdm
import h5py

In [29]:
import keras
from keras import Model
from keras.models import load_model
from keras.layers import Dense, Input, Masking, BatchNormalization, Layer, Embedding
from keras.layers import LSTM, Reshape, TimeDistributed, Concatenate, Multiply, RepeatVector
from keras.optimizers import Nadam
from keras.losses import sparse_categorical_crossentropy
from keras import backend as K

In [4]:
#change directory to project/parent directory
PROJECT_DIR = os.path.join(os.getcwd(), os.pardir)
os.chdir(PROJECT_DIR)

In [5]:
sys.path.insert(0, "src")

In [6]:
import helpers
helpers = reload(helpers)

In [7]:
from helpers import TextEncoder

In [8]:
CHARS_SEQS_PATH = "data/joke_char_sequences_Jan20.h5"
TOPICS_PATH = "data/joke_topics.pkl"
TOPIC_MODELER_PATH = "data/jokes_topic_modeler.pkl"
CHAR_DICT_PATH = "data/char_dict_Jan20.pkl"

In [9]:
BATCH_SIZE=512
#this is nubmer of 128 layers + 1 (there is layws at least one 127 (unit layers))

#i wonder if lstm is necasry of gru woruld work
MIDDLE_DEPTH=3
SEQ_LENGTH = 300
RESUME = True
MODEL_NAME = "rnn_jokes_topics_Jan25.hdf5"
#BASE_CELL_SIZE=64
#should change to maybe max seq length...
#for varios monitoring applications
MONITOR_FREQ=250

In [10]:
#easier to define here
TOTAL_DEPTH = MIDDLE_DEPTH + 4

# Load the Data

In [11]:
h5f = h5py.File(CHARS_SEQS_PATH, "r")
seqs = h5f["seqs"][:]
h5f.close()

In [12]:
len(seqs)

109095

In [85]:
num_seqs = seqs.shape[0]
num_seqs

109095

In [87]:
#average length
mean_length = 0.0
for seq in seqs:
    mean_length += len(seq)/num_seqs
print(mean_length)

344.61687520052584


In [14]:
#load char dict
pickle_in = open(CHAR_DICT_PATH,"rb")
char_dict = pickle.load(pickle_in)
pickle_in.close()

In [15]:
num_chars=len(char_dict)
print(num_chars)

98


In [16]:
#load encoder
topic_modeler = joblib.load(TOPIC_MODELER_PATH)

In [17]:
#load topics
topics = joblib.load(TOPICS_PATH)

In [18]:
topic_size = topics.shape[1]
print(topic_size)

32


In [88]:
#get top words per topic
topic_modeler.top_words(20)

[['say',
  'ask',
  'prostitute',
  'look',
  'leper',
  'tampon',
  'reply',
  'turn',
  'stick',
  'tip',
  'blonde',
  'fish',
  'nice',
  'think',
  'little',
  'pirate',
  'boy',
  'friend',
  'thing',
  'drive'],
 ['like',
  'coffee',
  'slave',
  'penis',
  'free',
  'chocolate',
  'life',
  'dark',
  'taste',
  'smell',
  'dick',
  'food',
  'look',
  'humor',
  'feel',
  'box',
  'lot',
  'sound',
  'beer',
  'domestic'],
 ['joke',
  'funny',
  'punchline',
  'dick',
  'laugh',
  'mom',
  'bad',
  'reddit',
  'apparent',
  'post',
  'r',
  'unemployed',
  'work',
  'chemistry',
  'wanna',
  'racist',
  'title',
  'long',
  'want',
  'execution'],
 ['lightbulb',
  'change',
  'feminist',
  'screw',
  'alzheimer',
  'funny',
  'dark',
  'juan',
  'mexicans',
  'basement',
  'patient',
  'pregnant',
  'room',
  'number',
  'germans',
  'beat',
  'efficient',
  'optometrist',
  'trick',
  'dead'],
 ['bar',
  'walk',
  'bartender',
  'drink',
  'beer',
  'serve',
  'order',
  'duck

# Generator

In [47]:
class CharGenSequence(object):
    def __init__(self, seqs, char_dict, topics, batch_size=500, seq_length=100):
        self.seqs = seqs
        self.char_dict = char_dict
        #need to know how big input to neural net (length of sequnece)
        self.seq_length = seq_length
        self.batch_size = batch_size
        #the  random permtuion returns a randomly sorted rangs
        self.seq_idxs = deque(np.random.permutation(len(seqs)).tolist())
        #now intitilize first batch
        #these are the indexes of seq_list that are used for the batch.
        #queu will help, can pop from left
        self.batch_idxs = [self.seq_idxs.pop() for _ in range(self.batch_size)]
        
        #ALWAYS START AT BEGINNING...
        self.seq_pos = [0 for seq in self.batch_idxs]
        
        #now topics
        self.topics = topics
        topic_dim = self.topics.shape[1]
        
        self.batch_topics = np.zeros((self.batch_size, topic_dim), dtype=np.float)
        
        #now fill
        for ix, batch_idx in enumerate(self.batch_idxs):
            self.batch_topics[ix, :] = self.topics[ix, :]
        
    def __next__(self):
        #make masks
        #used to determine if will use reset state or not...
        #will rely on brtaod casting to gie ii th eproer shape
        state_mask = np.ones((self.batch_size, 1), dtype=np.float32)
        #make x, the input a numpy array of zeros (initilaly)
        #nowing providing just indexes, since
        x = np.zeros((self.batch_size, self.seq_length, len(self.char_dict)), dtype=np.float)
        
        #will use sparse categorical
        y = np.zeros((self.batch_size, self.seq_length, 1), dtype=np.int32)
        
        #LOOP OVER BATCH
        #seq_idx is the index of the sequnce within seq_list
        #while batch_idx is the index of the sequence within the batch        
        for batch_idx, seq_idx in enumerate(self.batch_idxs):
            #work fowards...
            #GO UP TO LENGTH OF OUTPUTS...
            #check if this will be last batch for this input seq

            for pos_idx in range(self.seq_length):
                #self.seq_pos is start of sequence
                input_pos_idx =  self.seq_pos[batch_idx]+pos_idx
                #OUTPUTS ALWAYS ONE AHEAD OF INPUTS
                output_pos_idx = input_pos_idx + 1
                #if desired index does not exit, leave blank....
                try:
                    x[batch_idx, pos_idx, self.seqs[seq_idx][input_pos_idx]] = 1.
                except IndexError:
                    #leave at default of 0 (padding value)
                    pass
                try:
                    y[batch_idx, pos_idx, 0] = self.seqs[seq_idx][output_pos_idx]
                except IndexError:
                    #will be masked anyways?
                    y[batch_idx, pos_idx, 0] = self.char_dict["<BOUND>"]
        

            #DO SPECIAL STUFF IF length of sequence is less than than what was desired...
            if len(self.seqs[seq_idx]) <= (self.seq_length + self.seq_pos[batch_idx]):
                #first add back to avaialbe
                self.seq_idxs.append(self.batch_idxs[batch_idx])
                
                #pop left ensures first in, first out
                self.batch_idxs[batch_idx] = self.seq_idxs.popleft()
                    
                #set start pos back to 0
                self.seq_pos[batch_idx] = 0

                #get new word_indxs and wghts
                self.batch_topics[batch_idx, :] = self.topics[self.batch_idxs[batch_idx], :]
                
                #make masks = 0 to reset state
                #could probaly do this outside of the generator...
                state_mask[batch_idx,0] = 0.0
            else:
                #increment position by seq_length
                #want last output character to be the first input character...
                self.seq_pos[batch_idx]=self.seq_pos[batch_idx]+ self.seq_length
        return(x, y, self.batch_topics, state_mask)
    def __iter__(self):
        return self

In [48]:
#ACTUAL
gen_seq =  CharGenSequence(seqs, char_dict=char_dict, topics=topics, seq_length=SEQ_LENGTH, batch_size=BATCH_SIZE)

# Define Model

In [23]:
from helpers import sparse_softmax_cross_entropy_with_logits
from helpers import WghtdAverage

In [24]:
#this layer makes 
class Standardize(Layer):
    def __init__(self, **kwargs):
        super(Standardize, self).__init__(**kwargs)

    def compute_output_shape(self, input_shape):
        return input_shape
    
    def call(self, inputs, mask = None):
        #first, mean of 0
        inputs = inputs - K.mean(inputs, axis=-1, keepdims=True)
        #now, want to induce a variacne of 1
        inputs = inputs / (K.sqrt(K.mean(K.square(inputs), axis=-1, keepdims=True)) + K.epsilon())
        return inputs

In [25]:
def create_model(batch_size, input_length, num_chars, rnn_depth=1, topic_size=32):
    #character sequences
    character_input = Input(batch_shape=(batch_size,input_length, num_chars), dtype='float', name='char_indx_input')
    masked = Masking(name="mask")(character_input)
    #topic input
    topic_input = Input(batch_shape=(batch_size, topic_size), dtype='float', name='topic_input')
    topic = Standardize()(topic_input)
    topic_repeated = RepeatVector(input_length, name="repeat_topic")(topic)
    
    
    
    #first, to get to 32
    rnn =  LSTM(units=32,return_sequences=True, stateful=True, name="rnn0")(masked)
    rnn = sequences = BatchNormalization(name="normalize0")(rnn)
    #now 64
    rnn =  LSTM(units=64,return_sequences=True, stateful=True, name="rnn1")(rnn)
    #now concatenate topic..
    sequences = Concatenate(name="concatenate")([rnn, topic_repeated])
    sequences = BatchNormalization(name="normalize1")(sequences)
    
    for i in range(rnn_depth):
        rnn =  LSTM(units=128,return_sequences=True, stateful=True, name="rnn"+str(i+2))(sequences)
        sequences = BatchNormalization(name="normalize"+str(i+2))(rnn)

    #now 256 *maybe 512?
    #should have argument...
    rnn =  LSTM(units=256,return_sequences=True, stateful=True, name="rnn"+str(rnn_depth+2))(sequences)
    sequences = BatchNormalization(name="normalize"+str(rnn_depth+2))(rnn)
    rnn =  LSTM(units=512,return_sequences=True, stateful=True, name="rnn"+str(rnn_depth+3))(sequences)
        
    preds = TimeDistributed(Dense(num_chars), name="logits")(rnn)
    model = Model(inputs=[character_input, topic_input], outputs=preds)
    
    return model

In [32]:
if RESUME is not None:
    #this might not work since last layer is a different size
    #might have to load manually (for first...)
    training_model = load_model("models/"+MODEL_NAME, custom_objects={"Standardize":Standardize,
                                                                     "sparse_softmax_cross_entropy_with_logits":sparse_softmax_cross_entropy_with_logits})
    training_model.reset_states()
else:
    training_model = create_model(batch_size=BATCH_SIZE, input_length=SEQ_LENGTH, num_chars=num_chars,
                              rnn_depth=MIDDLE_DEPTH, topic_size=topic_size)
    training_model.compile(loss=sparse_softmax_cross_entropy_with_logits, optimizer=Nadam())

In [33]:
training_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_indx_input (InputLayer)    (512, 300, 98)       0                                            
__________________________________________________________________________________________________
mask (Masking)                  (512, 300, 98)       0           char_indx_input[0][0]            
__________________________________________________________________________________________________
rnn0 (LSTM)                     (512, 300, 32)       16768       mask[0][0]                       
__________________________________________________________________________________________________
topic_input (InputLayer)        (512, 32)            0                                            
__________________________________________________________________________________________________
normalize0

In [34]:
#make prediction model
#should have a dunciotn that extrracts, num_chars, rnn_depth and topic size from trianing model
#batch size of one, and only on time step
predict_model = create_model(batch_size=1, input_length=1, num_chars=num_chars,
                             rnn_depth=MIDDLE_DEPTH, topic_size=topic_size)

# Training Helpers

In [35]:
#function to sample an index from a probability array
def sample(preds, end_indx, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    #is logged proability
    #so exp(log(prob) / temperature) is smaller when temperature is higer
    #however derivative respect to temp: -log(prob) /temperature^2
    #-log(prob) is bigger when prob is smaller
    #so result is that lower temp makes smaller probs go to 0 faster
    preds = preds / temperature 
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [53]:
#NEED TO PROVIDE A "topic"
def generate_joke(model, topic, char_dict, max_len=1000, base_temp=0.5, temp_decay=0.1):
    model.reset_states()
    #prediction model only take 
    x_input = np.zeros((1,1, len(char_dict)), dtype =np.float32)
    
    #reverse the character dictionary, since will need to go from integer index to character
    char_dict_reverse = {value:key for key, value in char_dict.items()}
    #first input to the model will be the <BOUND> token
    x_input[0,0, char_dict["<BOUND>"]] = 1.0
    #x_indxs is used to store output, ant thus to generate jokes
    x_indxs = [char_dict["<BOUND>"]]
    
    for i in range(max_len):
        #want a decreasing temperature
        temperature= base_temp*exp(-i*temp_decay)
        #want only first...
        preds = model.predict_on_batch(x={"char_indx_input":x_input, "topic_input":topic})[0,0]
        next_index = sample(preds, end_indx=char_dict["<BOUND>"], temperature=1)
        #only need to update first index, since stateful...
        #make x_input again
        x_input = np.zeros((1,1, len(char_dict)), dtype =np.float32)
        x_input[0,0,next_index] = 1.0
        #now append to list that is used for text genration...
        x_indxs.append(next_index)
        if next_index == char_dict["<BOUND>"]:
            break
    x_tokens = [char_dict_reverse[indx] for indx in x_indxs]
    x_string = "".join(x_tokens)
    return(x_string)

In [93]:
generator_string = "how many stereotypes does it take to light bulb change"
generator_topic = topic_modeler.transform([generator_string])

In [41]:
def reset_states(model, layer, mask):
    states = model.get_layer(layer)._states
    states = [np.multiply(K.eval(state), mask) for state in states]
    model.get_layer(layer).reset_states(states)

# Train

In [52]:
#now a loop
epoch = 0
training_model.reset_states()
#need initial ys...
while True:
    epoch+=1
    print("***** Epoch {} *****".format(epoch)) 
    loss = np.zeros(MONITOR_FREQ,dtype=np.float32)
    for i in tqdm(range(MONITOR_FREQ)):
        char_input, y, topic_input, state_mask = next(gen_seq)
        loss[i] = training_model.train_on_batch(x={"char_indx_input":char_input, "topic_input":topic_input}, y=y)
        #now masking bit...
        for i in range(TOTAL_DEPTH):
            reset_states(training_model, "rnn"+str(i), state_mask)
    #checkpointer
    training_model.save("models/"+MODEL_NAME)
    print("Average loss of {:.4f}".format(np.mean(loss)))
    print("***** EXAMPLE OUTPUT*****")
    predict_model.set_weights(training_model.get_weights())
    joke = generate_joke(predict_model, generator_topic, char_dict, max_len=1000)
    print(joke)

***** Epoch 1 *****


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [36:21<00:00,  8.72s/it]


Average loss of 1.5741
***** EXAMPLE OUTPUT*****
<BOUND>"Haed brother guys giffed off tomeny... bit just new waitch the same behind, on a whosenseat walking in 2.

carion day came, put poilely of my approptoleing the tophernt, thenry good/to boy.<BOUND>
***** Epoch 2 *****


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [36:51<00:00,  8.84s/it]


Average loss of 1.3884
***** EXAMPLE OUTPUT*****
<BOUND>A fay catch the trice take

So the germanes! She says.
"I am in the cookh. As he asked the doctor." The priest, watches the store, and asks "Oh mistcum, my way."
The bartender gives a sign up and says "No, answer." 
The man says "Oh duck lady when it's confron, sit liveowi!" As a man again: "To want you with that it was there, it, so showed your word. I writped to here" He's long you are.." I help down the same and get his parpanet. So me, he snyed and peoss the last geod.  The cowboy flustorishes under her wine the body blowing rubbiwls using a beurind.
      the whole on Marie, "Two briate.  My chipf clubs her escar.  I here he poples on up that from house.
The brulip tree was selow. So we bet forgot a cornor," and scratcher couldn't life his face for an's statious is, whateve know Ch  rictos - Viche is Hellvon!
"Mr. chamas?" And ent given his wife's wraps a mednamy and made her spake, the next day he confused him for a refrige.

...

Average loss of 0.9974
***** EXAMPLE OUTPUT*****
<BOUND>Happy The Panther Beight was being vending was getting beaten moments.

His friends smile at the shaped arm and threw him. Is coming back to look for me it has a drink.<BOUND>
***** Epoch 39 *****


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [36:06<00:00,  8.67s/it]


Average loss of 0.9979
***** EXAMPLE OUTPUT*****
<BOUND>Some gorilla lingered porn?

MK. folks.<BOUND>
***** Epoch 40 *****


  2%|█▉                                                                                | 6/250 [00:51<35:06,  8.63s/it]

KeyboardInterrupt: 

In [111]:
wghts = predict_model.get_layer("rnn2").get_weights()

In [112]:
len(wghts)

3

In [113]:
wghts[0].shape

(96, 512)

In [114]:
wghts[1].shape

(128, 512)

In [115]:
wghts[2].shape

(512,)

In [118]:
wghts[2][64:96]

array([ 0.16500622,  0.07672426, -0.12786634, -0.00612003,  0.13232455,
       -0.12390276, -0.07230407, -0.15616518, -0.06531354, -0.09285146,
        0.14022237, -0.07699914,  0.70327419, -0.07669064,  0.15126373,
        0.0199868 , -0.14625588, -0.19004793, -0.27385962, -0.00417563,
       -0.07728644, -0.0612554 ,  0.04068965, -0.06528439,  0.04827083,
        0.03167893, -0.14356853,  0.12293271, -0.09711115,  0.03461432,
       -0.04996607, -0.1054599 ], dtype=float32)

In [117]:
wghts[2][0:32]

array([-0.05873594, -0.20834757, -0.19437824, -0.08044638, -0.13555704,
        0.0750309 , -0.016366  , -0.04769966, -0.14590505,  0.1670956 ,
       -0.01992786, -0.0447577 , -0.18429899, -0.02186309, -0.03105482,
        0.04352831,  0.06178947,  0.01678973, -0.03070449, -0.01133532,
       -0.08905493, -0.00231288,  0.11056606, -0.09005753, -0.26728538,
        0.02797053, -0.08320679, -0.08874695, -0.14318527, -0.20703405,
        0.0816564 ,  0.20222224], dtype=float32)